In [1]:
from obci_readmanager.signal_processing.read_manager import ReadManager
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal as ss
import pickle

Action points realized by this notebook:
- eeg signal upload from raw (bin), xlm, csv and tag files -> eeg dictionary with the option to call Fs, channel_names, rescaled values, tags
- filtering [lowpass - 13Hz, highpass - 8Hz, notch - 50Hz] -- filtering to leave only the miu band
- applying chosen montage; options: common_average, linked ears, chosen reference channel
- signal cutting for particular task describtion -> dictionary and signal save to local files

In [2]:
def download_signal(bin_file_path=None, xml_file_path=None, tag_file_path=None, csv_file_path=None):
    if bin_file_path is None:
        raise ValueError("'bin_file_path' is required.")

    if xml_file_path is None:
        raise ValueError("'xml_file_path' is required.")

    eeg = dict()

    mgr = ReadManager(xml_file_path, bin_file_path, tag_file_path if tag_file_path else None)

    # Retrieve the sampling frequency
    eeg['sampling'] = float(mgr.get_param("sampling_frequency"))

    # Retrieve the names of the channels
    eeg['channels_names'] = mgr.get_param("channels_names")

    # Retrieve EEG data and process it
    eeg['data'] = mgr.get_samples()
    eeg['data'] = eeg['data'] * 0.0715

    # Retrieve tags if the tag file is provided
    if tag_file_path:
        eeg['tags'] = mgr.get_tags()

    # Handle CSV file if provided
    if csv_file_path:
        try:
            eeg['data_csv'] = pd.read_csv(csv_file_path)
        except FileNotFoundError:
            print(f"Warning: CSV file not found at {csv_file_path}.")
            eeg['data_csv'] = None
    else:
        eeg['data_csv'] = None

    return eeg

In [3]:
def filter_signal(syg, sampling, lowpass_params=None, highpass_params=None, notch_params=None):
    # Default filter parameters
    default_lowpass = {'N': 2, 'Wn': 13, 'btype': 'lowpass', 'ftype': 'butter'}
    default_highpass = {'N': 2, 'Wn': 8, 'btype': 'highpass', 'ftype': 'butter'}
    default_notch = {'w0': 50, 'Q': 30}

    # Merge user-provided parameters with defaults
    lowpass_params = {**default_lowpass, **(lowpass_params or {})}
    highpass_params = {**default_highpass, **(highpass_params or {})}
    notch_params = {**default_notch, **(notch_params or {})}

    # Design lowpass filter
    sos_low = ss.iirfilter(
        N=lowpass_params['N'],
        Wn=lowpass_params['Wn'],
        btype=lowpass_params['btype'],
        ftype=lowpass_params['ftype'],
        fs=sampling,
        output='sos'
    )

    # Design highpass filter
    sos_high = ss.iirfilter(
        N=highpass_params['N'],
        Wn=highpass_params['Wn'],
        btype=highpass_params['btype'],
        ftype=highpass_params['ftype'],
        fs=sampling,
        output='sos'
    )

    # Design notch filter
    b_notch, a_notch = ss.iirnotch(
        w0=notch_params['w0'],
        Q=notch_params['Q'],
        fs=sampling
    )

    # Apply the filters sequentially
    syg = ss.sosfiltfilt(sos_low, syg, axis=-1)   # Lowpass filter
    syg = ss.sosfiltfilt(sos_high, syg, axis=-1)  # Highpass filter
    syg = ss.filtfilt(b_notch, a_notch, syg, axis=-1)  # Notch filter

    return syg

In [4]:
def apply_montage(syg, channels_names, montage_type='average', reference_channel=None):
    # Identify 'M1' and 'M2'
    unwanted_labels = {'M1', 'M2'}
    m1_m2_indices = [i for i, label in enumerate(channels_names) if label in unwanted_labels]

    # Remove 'M1' and 'M2' from channels
    indices_to_keep = [i for i, label in enumerate(channels_names) if label not in unwanted_labels]
    channels_names = [channels_names[i] for i in indices_to_keep]
    syg = syg[indices_to_keep]

    if len(m1_m2_indices) != 2:
        raise ValueError("Both 'M1' and 'M2' must be present in channels_names for average montage.")
    if montage_type == 'common_average':
        # Compute the mean
        average_reference = syg.mean(axis=0)

        # Apply montage
        syg = syg - average_reference

    elif montage_type == 'linked_ears':
        # Extract signals for 'M1' and 'M2'
        m1_signal = syg[m1_m2_indices[0], :]
        m2_signal = syg[m1_m2_indices[1], :]

        # Compute the mean of M1 and M2
        average_reference = (m1_signal + m2_signal) / 2

        # Apply montage
        syg = syg - average_reference

    elif montage_type == 'channel':
        if reference_channel is None:
            raise ValueError("`reference_channel` must be specified for montage_type='channel'.")

        # Find the reference channel index
        if reference_channel not in channels_names:
            raise ValueError(f"Reference channel '{reference_channel}' not found in channels_names.")

        ref_index = channels_names.index(reference_channel)
        ref_signal = syg[ref_index, :]

        # Apply montage
        syg = syg - ref_signal

    else:
        raise ValueError("Invalid montage_type. Choose 'average' or 'channel'.")

    return syg, channels_names

In [5]:
def cut_signal(syg, tags, sampling, amplitude_limit=500):
  # Initialize parameters
  dlugosc = int((2 + 6) * sampling)
  lewa_list = []
  prawa_list = []
  lewa, prawa = ('lewa', 'prawa')

  for tag in tags:
      # Determine if the tag is frequent or rare based on the timestamp
      t0 = int(sampling * (tag['start_timestamp'] - 2))

      # Slice the signal segment
      segment = syg[:, t0:t0 + dlugosc]

      if tag['desc']['strona'] == lewa:
          lewa_list.append(segment)
      elif tag['desc']['strona'] == prawa:
          prawa_list.append(segment)

  lewa_array = np.array(lewa_list)

  prawa_array = np.array(prawa_list)

  return lewa_array, prawa_array

In [6]:
for signal_type in ["ruch", "wyobrazenie"]:

    name = "mati"
    xml_file = f"{name}/{name}_{signal_type}.obci.xml"
    bin_file = f"{name}/{name}_{signal_type}.obci.raw"
    csv_file_path = f"{name}/{name}_{signal_type}.csv"
    tag_file_path = f"{name}/{name}_{signal_type}.obci.tag"

    EEG_mati = download_signal(bin_file, xml_file, tag_file_path, csv_file_path)
    EEG_mati['data'], EEG_mati['channels_names'] = apply_montage(EEG_mati['data'], EEG_mati['channels_names'], 'common_average')
    EEG_mati['data'] = filter_signal(EEG_mati['data'], EEG_mati['sampling'])

    EEG_mati['lewa'], EEG_mati['prawa'] = cut_signal(EEG_mati['data'], EEG_mati['tags'], EEG_mati['sampling'])
    merged = np.stack((EEG_mati['lewa'], EEG_mati['prawa']), axis=0)

    saved = 0
    if not saved:
        np.save(f"{name}_{signal_type}_dane_miu.npy", merged)
        with open(f"{name}_{signal_type}_EEG_miu.pkl", "wb") as f:
            pickle.dump(EEG_mati, f)

2025-04-08 22:37:08,648 - data_source - INFO - All data set requested for the first time. Start reading all data from the file...
2025-04-08 22:37:11,105 - data_source - INFO - All data set requested for the first time. Start reading all data from the file...
